Sentiment Analysis about facebook political singlish comments

In [9]:
#import Libraries which need
import nltk
import pandas as pd
import os

In [10]:
#from Natural language toolkit import which need
from nltk import FreqDist
from nltk.tokenize import word_tokenize

In [11]:
#add & read the comments excel file 
df = pd.read_excel(os.path.join("comments_e.xlsx"),engine='openpyxl',)
    
negative_df = df[df["Annotate"]=="Negative"]
positive_df = df[df["Annotate"]=="Positive"]

negative_comments = negative_df["Comment"]   
positive_comments = positive_df["Comment"]

In [12]:
#define probability function calculation with bigram,unigram
def probability(comment_bigrams, corpus_comment):
    unigram_list = []
    bigram_list = []

    for line in corpus_comment:
        line = "< " + line + " >"
        nltk_tokens = nltk.word_tokenize(line.lower())
        unigram_list = unigram_list + list(nltk_tokens)
        bigrams = nltk.bigrams(nltk_tokens)
        bigram_list = bigram_list + list(bigrams)
        
    unigram_freq = FreqDist(unigram_list)
    bigram_freq = FreqDist(bigram_list)

    probability = 1.0
    for x in range(len(comment_bigrams)):
        if comment_bigrams[x] in bigram_freq:
            word2_count_bigram = bigram_freq[comment_bigrams[x]]
        else:
            word2_count_bigram = 0

        if comment_bigrams[x][0] in unigram_freq:
            word1_count_bigram = unigram_freq[comment_bigrams[x][0]]
        else:
            word1_count_bigram = 0
        
        #can depend on corpus total word count
        unigram_vocabulary_count = 500
        probability = probability * ((word2_count_bigram + 1) / (word1_count_bigram + unigram_vocabulary_count))

    return probability

In [13]:
#define perplexity function
def perplexity(probability, n):
    return pow(probability, (-1 / n))

In [14]:
#calculation of probability
def output_testing(comment):
    
    print('Entered comment:', comment)
    nltk_tokens = nltk.word_tokenize(comment.lower())
    number_of_words = len(nltk_tokens)
    comment_bigrams = list(nltk.bigrams(nltk_tokens))

    negative_probability = probability(comment_bigrams, negative_comments)
    positive_probability = probability(comment_bigrams, positive_comments)

    print("Negative probability = " + str(negative_probability) + ", Positive probability = " + str(positive_probability))

    #sentiment analysis
    
    if (negative_probability > positive_probability):
        print("Sentiment : Negative comment")
        pp = perplexity(negative_probability, number_of_words)
        print("Perplexity = " + str(pp))
    elif negative_probability < positive_probability:
        print("Sentiment : Positive comment")
        pp = perplexity(positive_probability, number_of_words)
        print("Perplexity = " + str(pp))
    else:
        print("Cannot classify")

In [15]:
#testing (add testing comment as '<comment>')

output_testing('Hoda adarsamath wedak ')

Entered comment: Hoda adarsamath wedak 
Negative probability = 3.98406374501992e-06, Positive probability = 4e-06
Sentiment : Positive comment
Perplexity = 62.996052494743644


In [16]:
#testing (add testing comment as '<comment>')

output_testing('aparade ape watina chandaya')

Entered comment: aparade ape watina chandaya
Negative probability = 6.349080274612323e-08, Positive probability = 8e-09
Sentiment : Negative comment
Perplexity = 62.9973520808422


In [8]:
#testing (add testing comment as '<comment>')

output_testing('Anieaarenma Ranjan Ramanayaka mahathataa .jayen jayama wewaaaaa. Jesu pihitai.')

Entered comment: Anieaarenma Ranjan Ramanayaka mahathataa .jayen jayama wewaaaaa. Jesu pihitai.
Negative probability = 9.999570330962344e-28, Positive probability = 1.0240000000000004e-27
Sentiment : Positive comment
Perplexity = 284.190198160869
